DATA MINING FILM SALES

1. Dự đoán doanh thu từng thể loại phim nếu mỗi thể loại có tổng lượt chiếu ở từng rạp là 35 suất

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression


In [5]:
df1 = pd.read_excel("film_mining.xlsx", sheet_name= "Câu 1")

mapping = {'Cinema.Cinema Name': 'Cinema Name','City.City Name': 'City Name',
           'Film Cate.Film Category': 'Film Category',
           'Film Sales Count':'total screening capacity'}
df1 = df1.rename(columns= mapping)
df1.head()


,Cinema Name,City Name,Film Category,total screening capacity,Total Sales
0,BHD Cinema,Ha Noi,Action,60,2698130
1,BHD Cinema,Ha Noi,Animation,28,970310
2,BHD Cinema,Ha Noi,Kid,17,643251
3,CGV Cinema,Ho Chi Minh,Action,37,3011090
4,CGV Cinema,Ho Chi Minh,Animation,60,4928320


In [8]:
df1['Revenue_Per_Screening'] = df1['Total Sales'] / df1['total screening capacity']
features = ['total screening capacity']
target = 'Revenue_Per_Screening'
model = LinearRegression()
train_data = df1[features + [target]]

model.fit(train_data[features], train_data[target])

df1['Predicted_Revenue'] = model.predict(df1[features] * 35)

print(df1[['Cinema Name', 'Film Category', 'Predicted_Revenue']])

      Cinema Name Film Category  Predicted_Revenue
0      BHD Cinema        Action      377870.362829
1      BHD Cinema     Animation      200018.162239
2      BHD Cinema           Kid      138881.468286
3      CGV Cinema        Action      250039.093655
4      CGV Cinema     Animation      377870.362829
5      CGV Cinema           Kid      133323.587018
6        Cinestar        Action       72186.893065
7        Cinestar           Kid      111092.061944
8   Galaxy Cinema        Action      733574.764009
9   Galaxy Cinema     Animation      755806.289082
10  Galaxy Cinema           Kid      472354.344392
11    Mega Cinema        Action       66629.011796


2. Dự đoán giá vé của BHD Cinema nếu mỗi loại ghế giá vé sẽ tăng 5 đồng

In [16]:
df2 = pd.read_excel("film_mining.xlsx", sheet_name= "Câu 2")

mapping = {'Cinema.Cinema Name': 'Cinema Name','Cinema.Standard Seats Price': 'Standard Seats Price','Cinema.VIP Seats Price': 'VIP Seats Price','DATE MASTER.Month':'Month'}
df2 = df2.rename(columns= mapping)
df2['Standard Seats Use'] = df2['Standard Seats Sold']- df2['Standard Seats Out']
df2['VIP Seats Use'] = df2['VIP Seats Sold'] - df2['VIP Seats Out']
df2.head()


,Cinema Name,Standard Seats Price,VIP Seats Price,Standard Seats Out,Standard Seats Sold,VIP Seats Out,VIP Seats Sold,Month,Standard Seats Use,VIP Seats Use
0,BHD Cinema,25,45,3,1196,16,569,10,1193,553
1,BHD Cinema,25,45,0,895,22,2018,3,895,1996
2,BHD Cinema,25,45,3,1319,7,779,4,1316,772
3,BHD Cinema,25,45,28,9264,84,7595,5,9236,7511
4,BHD Cinema,25,45,35,13233,115,9885,6,13198,9770


In [27]:
current_bhd_standard_seats_use = df2[df2['Cinema Name'] == 'BHD Cinema']['Standard Seats Use'].sum()
current_bhd_vip_seats_use = df2[df2['Cinema Name'] == 'BHD Cinema']['VIP Seats Use'].sum()

features = ['Standard Seats Price']
target_standard_seats_use = 'Standard Seats Use'
target_vip_seats_use = 'VIP Seats Use'

df_bhd = df2[df2['Cinema Name'] == 'BHD Cinema']

model_bhd_standard_seats = LinearRegression()
model_bhd_standard_seats.fit(df_bhd[features], df_bhd[target_standard_seats_use])

model_bhd_vip_seats = LinearRegression()
model_bhd_vip_seats.fit(df_bhd[features], df_bhd[target_vip_seats_use])

price_increase = 5
prediction_data = pd.DataFrame({
    'Standard Seats Price': [df_bhd['Standard Seats Price'].iloc[0] + price_increase],
})

predicted_bhd_standard_seats_use = model_bhd_standard_seats.predict(prediction_data)[0]
predicted_bhd_vip_seats_use = model_bhd_vip_seats.predict(prediction_data)[0]

# Tính toán doanh thu dự kiến
new_predicted_revenue = predicted_bhd_standard_seats_use * prediction_data['Standard Seats Price'].iloc[0] + predicted_bhd_vip_seats_use * (df_bhd['VIP Seats Price'].iloc[0] + price_increase)

print(f'Dự đoán số lượt vé Standard được bán ra của rạp BHD Cinema sau khi tăng giá là: {predicted_bhd_standard_seats_use}')
print(f'Dự đoán số lượt vé VIP được bán ra của rạp BHD Cinema sau khi tăng giá là: {predicted_bhd_vip_seats_use}')
print(f'Dự đoán doanh thu của rạp BHD Cinema sau khi tăng giá là: {new_predicted_revenue}')


Dự đoán số lượt vé Standard được bán ra của rạp BHD Cinema sau khi tăng giá là: 5071.375
Dự đoán số lượt vé VIP được bán ra của rạp BHD Cinema sau khi tăng giá là: 4227.75
Dự đoán doanh thu của rạp BHD Cinema sau khi tăng giá là: 363528.75
